# 4. Tiedon vertailu

## 4.1 Geometris-topologinen vertailu

### 4.1.1 Kiinteistöpalstat ja kaavaindeksit

<span style="color:red">**Tämä vaihe ei ole pakollinen!**</span>

MML:n kiinteistöpalstojen ja kaavaindeksien vertailussa tuodaan esiin, miten hyvin kaavaindeksien rajat yhtyvät kiinteistörajoihin. Topologinen yhtenevyys ilmaistaan prosenttiarvona (iou_1), joka lasketaan yksittäisen kiinteistöpalstan ja kaavaindeksin risteyskohdan (intersection) pinta-alan suhteena kiinteistöpalstan pinta-alaan.

Laskenta palauttaa kaikki ne palstageometriat, jotka sivuavat vähintään yhtä kaavaindeksiä.

In [ ]:
# Käytettävien Python moduulien ja kehitettyjen funktioiden sisäänluku
import geopandas as gpd
from lib.palsta_kaavaindeksi_vertailu import calculateIntersectionPercentage
from lib.accessory_functions import readPickleData, saveGPKG

In [ ]:
# Vertailuun tuleva kaava-aineisto ja MML:n kiinteistöpalsta-aineisto
palstat = readPickleData(inputfp=r"<insert filepath here>.pkl")
kaavadata = gpd.read_file(r"<insert filepath here>.gpkg", layer="<insert layer name here>")

# Ajo on suositeltavaa toteuttaa kunta kerrallaan
kunta = kaavadata.loc[kaavadata['kuntanimi'] == '<insert kuntanimi here>']

In [ ]:
# Laskenta
palstat_calc = calculateIntersectionPercentage(kaava=kunta,
                                               kaava_nro_column="<insert column name here>", # kaavaindeksin yksilöivä tieto, esim. kaavatunnus
                                               palstat=palstat)

# Tallennus tarvittaessa
#saveGPKG(palstat_calc, outputfp=r"<insert filepath here>.gpkg", layer_name="<insert layer name here>")

### 4.1.2 Dimensionally Extended 9-Intersection Model (DE-9IM)

KTJ:n ja kuntien kaava-aineistojen vertailu perustuu ETL-työkalussa [OGC:n DE-9IM -standardin mukaiseen malliin](https://postgis.net/workshops/postgis-intro/de9im.html), jonka avulla tunnistetaan yhtenevät, päällekkäiset sekä toisistaan eriävät geometriat. Tämän lisäksi ETL-muunnin vertailee kaavarajojen muotoa ja pinta-aloja automaattisesti.

DE-9IM-mallissa kahden geometrian välinen relaatio ilmaistaan 9-merkkisellä mallilla. Malli voi olla esimerkiksi muotoa:

| DE-9IM | Sisusta (interior) | Raja (boundary) | Ulkoraja (exterior) |
| :--- | :--- | :--- | :--- |
| Sisusta (interior) | 2 | 1 | 2 |
| Raja (boundary) | 1 | 0 | 1 |
| Ulkoraja (exterior) | 2 | 1 | 2 |


Mallia luetaan seuraavasti:

- 2: Geometrioiden risteyskohta on alue (Polygon) eli kaksiulotteinen geometria
- 1: Geometrioiden risteyskohta on viiva (LineString) eli yksiulotteinen geometria
- 0: Geometrioiden risteyskohta on piste (Point) eli nollaulotteinen geometria

Muita DE-9IM-mallissa esiintyviä arvoja voivat olla:

- F: False - risteyskohtaa ei esiinny
- T: True - mikä tahansa ei-false arvo on sallittu (0,1 tai 2)
- *: kaikki sallitaan

**Jos kahden geometrian välinen relaatiomalli vastaa muotoa "T*F**FFF*", geometriat ovat täydellisesti samat!** Tämä on hyvin harvinaista, sillä DE-9IM-malli ei salli pienintäkään murto-osan eroavaisuutta. <span style="color:red">Tämän takia ETL-muunnin tulkitsee kaavat geometris-topologisesti yhteneviksi, jos niiden topologinen tarkkuus on vähintään 98 %</span>

Geometris-topologinen tarkkuus ilmaistaan ETL-muuntimessa iou-arvolla. Lyhenne tulee sanoista *intersection of union*, geometrioiden yhtymäkohdan risteyskohta. Mikäli kaksi kaavaa ovat geometris-topologisesti tismalleen samanlaiset, niiden risteyskohdan sekä yhtymäkohdan pinta-alat ovat tismalleen samat. Näiden suhde on täten 100 %. Mikäli kaavoissa on geometris-topologista eroavaisuutta, niiden yhtymäkohta on erisuuri kuin risteyskohta. Tällöin yhtymäkohdan ja risteyskohdan pinta-alojen suhde eroaa 100 %:sta

Geometris-topologista eheyttä ilmaistaan ETL-työkalussa kaiken kaikkiaan seuraavin kentin:

| Ominaisuustieto | Selite |
| :--- | :--- |
| area_ha | Kaavan pinta-ala hehtaareina. |
| refe_area_ha | Vastaavan kaavan pinta-ala hehtaareina. |
| de9im_pattern | DE-9IM-mallin mukainen relaatio 9-merkkisellä mallilla. |
| topo_equal | Boolean-tieto. Onko vertailtavat kaavat topologisesti yhtenevät vai ei. <span style="color:red">Kyllä, jos iou-arvo on vähintään 98.</span> |
| iou | Intersection of union -prosentti. Ilmaistaan vertailtavien kaavojen yhtymä- ja risteyskohdan suhteena. |
| refe_kaavatunnus | Vastaavan kaavan kaavatunnus. |
| a_delta_% | Vertailtavien kaavojen pinta-alojen suhteellinen virheprosentti. |

### 4.1.3 KTJ:n ja kunta-aineistojen kaavat (DE-9IM)

Kuntien ja KTJ:n kaava-aineistojen välillä ei ole yksilöivää tunnistetta. Täten ominaisuustietojen pohjalta ei voida tietää, mikä kunnan kaava vastaa KTJ-kaavaa ja päin vastoin. ETL-työkalussa ongelma on ratkaistu siten, että kaavan vastine toisessa datasetissa on se kaava, jonka iou-prosentti on suurin.

Geometris-topologinen vertailu on mahdollista toteuttaa ETL-työkalulla joko kunta- tai KTJ-aineistopohjaisesti. **On kuitenkin suositeltavaa, että KTJ-aineistoa käytetään vertailun pohjana, sillä aineisto on lähtökohtaisesti eheämpää!**

<span style="color:red"><ins>**TÄRKEÄÄ TIETOA KAAVOISTA JA KAAVAINDEKSEISTÄ**</ins></span>

KTJ:n kunta-aineistot on aina rivitetty indeksitasolle, joten ETL-työkalu osaa tehdä KTJ:n kaavoille kaavatunnukseen pohjautuvan dissolven (=kaavaindeksien yhdistäminen kaavaksi) automaattisesti. Näin ei kuitenkaan aina ole kuntien kaava-aineistoissa, vaan yksi kunta-aineiston rivi voi edustaa kaavaa kokonaisuudessaan. Itse vertailu halutaan aina toteuttaa kaavatasolla. <span style="color:red">Jos kunta-aineisto on indeksitasolla, sille tulee tehdä dissolve kaavatunnukseen tai muuhun kaavan yksilöivään tietoon pohjautuen!</span>

Näissä tilanteissa KTJ-pohjaisessa vertailussa vertailufuntiolle annetaan valinnaiset parametrit:

- dissolve_refe=True
- dissolve_column="kaavan yksilöivän kentän nimi"

Kuntapohjaisen vertailun vastaavat parametrit ovat:

- dissolve_kunta=True
- dissolve_column="kaavan yksilöivän kentän nimi"

## 4.2 Ominaisuustietojen vertailu

Siinä missä geometris-topologinen vertailu on mahdollista tehdä myös kunta-aineistoon pohjautuen, **ominaisuustietojen vertailu on toteutettu VOOKA-pilotissa vain KTJ-pohjaisesti**. KTJ- ja kunta-aineiston ominaisuustietojen yhdenmukaisuutta ilmaistaan ETL-työkalussa kaiken kaikkiaan seuraavin kentin:

| Ominaisuustieto | Selite |
| :--- | :--- |
| kl_equal | Boolean-tieto. Onko KTJ-aineiston kaavalaji-tieto sama kuin kunta-aineistossa. |
| hyv_equal | Kertoo, onko KTJ- ja kunta-aineiston hyväksymispäivämäärä sama. |
| voim_equal | Kertoo, onko KTJ- ja kunta-aineiston voimaantulopäivämäärä sama. |

Ensimmäinen ominaisuustietokenttä “kl_equal” kertoo boolean-tietona onko vertailtavien aineistojen kaavalajit samat. Toinen ja kolmas ominaisuustietokenttä “hyv_equal” sekä “voim_equal” ilmaisevat hyväksymis- ja voimaantulopäivämäärien yhteneväisyyden sekä niihin liittyvät mahdolliset puutteet. Kentissä voi esiintyä:

- “Hyvpvm/voimpvm puuttuu molemmista”
- “Hyvpvm/voimpvm puuttuu KTJ”
- “Hyvpvm/voimpvm puuttuu kunta”
- 1 - Tarkasteltavat päivämäärät ovat samat
- 0 - Tarkasteltavat päivämäärät eroavat toisistaan

Vahvistamispäivämääriä ei ole mukana vertailussa, sillä ne puuttuvat KTJ-aineistosta kokonaan.

## 4.3 Toteutus

ETL-työkalussa geometris-topologinen vertailu sekä ominaisuustietojen rinnastus on rakennettu samaan ajoon.

In [ ]:
# Kehitettyjen funktioiden sisäänluku
from lib.kunta_ktj_kaavavertailu import compareKuntadataToKTJ
from lib.compare_ktj_to_kunta_main import compareKTJindicesToKunta
from lib.accessory_functions import setupMasterGDF, appendDataToMaster, saveGPKG

In [ ]:
import geopandas as gpd

# Vertailtavat kaava-aineistot
# Vertailtavat kaava-aineistot
kunta_data = gpd.read_file(r"<insert filepath here>.gpkg", layer="<insert layer name here>")
ktj_data = gpd.read_file(r"<insert filepath here>.gpkg", layer="<insert layer name here>")
                         
                         
"""
# Kuntapohjainen esimerkki, tulosten mukana ei tule ominaisuustietovertailua
tulokset = compareKuntadataToKTJ(kunta_data=kunta_data,
                                 ktj_data=ktj_data,
                                 kuntanimi='Sulkava',
                                 kaavalajit=['31', '39'],
                                 dissolve_kunta=True,
                                 dissolve_column='kaavatunnus')
"""

# Vertailu, kunta kerrallaan
# Lue kunta_ktj_kaavavertailu.py -tiedostosta funktion docstring, jotta ymmärrät parametreja paremmin!
# Lisää "dissolve_refe", ja "dissolve_column" parametrit tarvittaessa
results = compareKTJindicesToKunta(base_kaavadata=ktj_data,
                                   refe_kaavadata=kunta_data,
                                   kuntanimi='<insert kuntanimi here>',
                                   kaavalajit=["<list kaavalaji numbers here>"]) # Asemakaavoilla esim. ['31', '39'], rantakaavoilla ['33'] jne.
                                   #dissolve_refe=True,
                                   #dissolve_kunta=True,
                                   #dissolve_column='kaavatunnus') # Asemakaavoilla esim. ['31', '39'], rantakaavoilla ['33'] jne.


# Aja vain kerran ennen ensimmäistä appendia! 
master = setupMasterGDF(data=results, geom_column='geometry')



try:
    master = appendDataToMaster(master_data=master, append_data=results)
except SystemExit:
    results = results[list(master.columns)]
    master = appendDataToMaster(master_data=master, append_data=results)

master = results

# Tallennus
saveGPKG(master, outputfp=r"<insert filepath here>", layer_name="<insert layer name here>")